# Gemeinde Abfrage -> Liste der Elcom's

In [ ]:
import os
import requests
import csv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


gemeinden = []
with open('../data/vese/Gemeindestand.csv', 'r') as file:
    reader = csv.reader(file, delimiter=';')  # Verwende ; als Trennzeichen
    next(reader)  # Überspringe die Kopfzeile
    for row in reader:
        gemeinden.append(row[4])

# Lizenzschlüssel (API-Key)
license_key = 'vkmshstyf3r9epknhajqhenzog4opv6wcajd7ob7'

# Funktion zur Abfrage der Gemeinde-API mit Retry-Mechanismus
def abfrage_gemeinde(idofs, license_key):
    url = f'https://opendata.vese.ch/pvtarif/api/getData/muni?idofs={idofs}&licenseKey={license_key}'
    
    # Retry-Mechanismus konfigurieren
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    
    try:
        response = session.get(url, timeout=20)
        if response.status_code != 200:
            print(f"Fehler bei der Abfrage für {idofs}: {response.status_code}")
            return {'valid': False}
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Abfrage für {idofs}: {e}")
        return {'valid': False}

# Ergebnisse speichern
ergebnisse = []

for index, gemeinde in enumerate(gemeinden, start=1):
    if index % 100 == 0:  # Status nur bei jedem 100. Gemeinde ausgeben
        progress = (index / len(gemeinden)) * 100
        print(f"Verarbeite Gemeinde {index} von {len(gemeinden)} ({progress:.2f}%)")
    result = abfrage_gemeinde(gemeinde, license_key)
    if result['valid']:
        for evu in result['evus']:
            ergebnisse.append([evu['nrElcom'], evu['Name'], evu['idofs'], evu['PLZ'], evu['Gemeinde']])  

# Datei löschen, wenn sie existiert
output_file = '../data/vese/gemeinde_ergebnisse.csv'
if os.path.exists(output_file):
    os.remove(output_file)

# Ergebnisse in eine CSV-Datei schreiben
with open('../data/vese/gemeinde_ergebnisse.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['nrElcom', 'Name', 'idofs', 'PLZ', 'Gemeinde'])
    writer.writerows(ergebnisse)


# Abfgrage der Elcom's

In [ ]:
import csv

elcoms = set()  # Verwende ein Set, um Duplikate zu vermeiden
with open('../data/vese/gemeinde_ergebnisse.csv', 'r') as file:
    reader = csv.reader(file, delimiter=';')  # Verwende ; als Trennzeichen
    next(reader)  # Überspringe die Kopfzeile
    for row in reader:
        elcoms.add(row[0])  # Füge Werte zum Set hinzu

# Optional: Wenn du wieder eine Liste benötigst
elcoms = list(elcoms)
print(f"Anzahl ElComs: {len(elcoms)}")

# Lizenzschlüssel (API-Key)
license_key = 'vkmshstyf3r9epknhajqhenzog4opv6wcajd7ob7'

# Array von Jahreszahlen von 2014 bis 2024
years = list(range(2014, 2025))  # range(start, stop), stop ist exklusiv

# Funktion zur Abfrage der Gemeinde-API mit Retry-Mechanismus
def abfrage_ew(evuId, year, license_key):
    year_short = str(year)[-2:]  # Nur die letzten zwei Ziffern der Jahreszahl
    url = f'https://opendata.vese.ch/pvtarif/api/getData/evu?evuId={evuId}&year={year_short}&licenseKey={license_key}'
    
    # Retry-Mechanismus konfigurieren
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    
    try:
        response = session.get(url, timeout=20)
        if response.status_code != 200:
            print(f"Fehler bei der Abfrage für {evuId}, Jahr {year}: {response.status_code}")
            return {'valid': False}
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Abfrage für {evuId}, Jahr {year}: {e}")
        return {'valid': False}

# Ergebnisse speichern
ergebnisse = []

for year_index, year in enumerate(years, start=1):
    print(f"Verarbeite Jahr {year_index} von {len(years)}: {year}")
    for elcom_index, elcom in enumerate(elcoms, start=1):
        if elcom_index % 100 == 0:  # Status nur bei jedem 100. ElCom ausgeben
            print(f"  Verarbeite ElCom {elcom_index} von {len(elcoms)}")
        result = abfrage_ew(elcom, year, license_key)
        if result['valid']:
            evu_data = {
                'year': year,
                'nomEw': result.get('nomEw'),
                'nrElcom': result.get('nrElcom'),
                'link': result.get('link'),
                'tarif1': result.get('tarif1'),
                'tarif2': result.get('tarif2'),
                'tarif3': result.get('tarif3'),
                'explText': result.get('explText'),
                'counterCost': result.get('counterCost'),
                'loadCurveCost': result.get('loadCurveCost'),
                'nbrPowerCat': result.get('nbrPowerCat'),
                'datasheet': result.get('datasheet'),
                'nbrCustCat': result.get('nbrCustCat'),
                'energyProduct': result.get('energyProduct'),
                'minAutoConsumption': result.get('minAutoConsumption'),
                'maxAutoConsumption': result.get('maxAutoConsumption'),
                'compModel': result.get('compModel'),
                'thirdPartyComp': result.get('thirdPartyComp'),
                'additionalFees': result.get('additionalFees'),
                'conditions': result.get('conditions'),
                'contactPerson': result.get('contactPerson'),
                'contactMail': result.get('contactMail'),
                'contactPhone': result.get('contactPhone')
            }

            # Leistungskategorien auslesen
            for i in range(1, int(result.get('nbrPowerCat', 0)) + 1):
                evu_data[f'power{i}'] = result.get(f'power{i}')
                evu_data[f'energy{i}'] = result.get(f'energy{i}')
                evu_data[f'eco{i}'] = result.get(f'eco{i}')
                evu_data[f'energyAuto{i}'] = result.get(f'energyAuto{i}')
                evu_data[f'ecoAuto{i}'] = result.get(f'ecoAuto{i}')

            # Kundenkategorien auslesen
            for i in range(1, int(result.get('nbrCustCat', 0)) + 1):
                evu_data[f'custCat{i}'] = result.get(f'custCat{i}')
                evu_data[f'comp{i}'] = result.get(f'comp{i}')
                evu_data[f'compAuto{i}'] = result.get(f'compAuto{i}')

            ergebnisse.append(evu_data)  

# Datei löschen, wenn sie existiert
output_file = '../data/vese/evu_ergebnisse.csv'
if os.path.exists(output_file):
    os.remove(output_file)

# Ergebnisse in eine CSV-Datei schreiben
with open('../data/vese/evu_ergebnisse.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(ergebnisse[0].keys())  # Schreibe die Header basierend auf den Schlüsseln des ersten Ergebnisses
    writer.writerows([evu.values() for evu in ergebnisse])  # Schreibe die Werte der Ergebnisse